### Instrucciones para Descargar los Datos desde Kaggle

Para descargar los archivos de datos de la competencia de Kaggle directamente a este entorno de Google Colab, sigue los siguientes pasos:

1. **Generar el archivo `kaggle.json`**:
   - Ve a [Kaggle](https://www.kaggle.com).
   - Inicia sesión con tu cuenta.
   - Dirígete a tu perfil (haciendo clic en tu imagen de perfil en la esquina superior derecha) y selecciona **Settings**.
   - Desplázate hasta la sección **API** y haz clic en **Create New API Token**.
   - Esto descargará un archivo `kaggle.json` en tu computadora. Este archivo contiene las credenciales necesarias para acceder a la API de Kaggle.

2. **Subir el archivo `kaggle.json` a Google Colab**:
   - Ejecuta la celda que te pide subir archivos.
   - Selecciona el archivo `kaggle.json` que descargaste de Kaggle.

3. **Ejecutar el código para descargar los datos**:
   - Una vez que hayas subido el archivo `kaggle.json`, las siguientes celdas descargarán y descomprimirán automáticamente los archivos de la competencia desde Kaggle.

**Subir el archivo `kaggle.json`**:

In [ ]:
!pip install kaggle

# Solicitar el archivo kaggle.json
from google.colab import files
files.upload()  # Subir el archivo kaggle.json

# Crear el directorio .kaggle y mover el archivo kaggle.json
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json


**Código para descargar los datos**:


In [ ]:
# Descargar los datos
!kaggle competitions download -c udea-ai4eng-20242

# Descomprimir los archivos descargados
!unzip udea-ai4eng-20242.zip

 75% 15.0M/20.1M [00:00<00:00, 150MB/s]
100% 20.1M/20.1M [00:00<00:00, 146MB/s]
Archive:  udea-ai4eng-20242.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


**Importando librerías**

In [ ]:
# Importar librerías
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer

# **Carga y visualización del dataset**

In [ ]:
# Cargar los datos
data = pd.read_csv('train.csv')

# Mostrar los primeros registros
data.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_EDUCACIONMADRE,ESTU_PAGOMATRICULAPROPIO,RENDIMIENTO_GLOBAL
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Postgrado,No,medio-alto
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Técnica o tecnológica incompleta,No,bajo
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Secundaria (Bachillerato) completa,No,bajo
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Secundaria (Bachillerato) completa,No,alto
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Primaria completa,No,medio-bajo


**Información del dataset**

In [ ]:
# Mostrar tipos de datos y valores nulos
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 692500 entries, 0 to 692499
Data columns (total 12 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   ID                              692500 non-null  int64 
 1   PERIODO                         692500 non-null  int64 
 2   ESTU_PRGM_ACADEMICO             692500 non-null  object
 3   ESTU_PRGM_DEPARTAMENTO          692500 non-null  object
 4   ESTU_VALORMATRICULAUNIVERSIDAD  686213 non-null  object
 5   ESTU_HORASSEMANATRABAJA         661643 non-null  object
 6   FAMI_ESTRATOVIVIENDA            660363 non-null  object
 7   FAMI_TIENEINTERNET              665871 non-null  object
 8   FAMI_EDUCACIONPADRE             669322 non-null  object
 9   FAMI_EDUCACIONMADRE             668836 non-null  object
 10  ESTU_PAGOMATRICULAPROPIO        686002 non-null  object
 11  RENDIMIENTO_GLOBAL              692500 non-null  object
dtypes: int64(2), object(10)
memory

**Identificar valores faltantes**

In [ ]:
# Explorar la cantidad de valores faltantes por columna
missing_values = data.isnull().sum()
print("Valores faltantes:\n", missing_values)

Valores faltantes:
 ID                                    0
PERIODO                               0
ESTU_PRGM_ACADEMICO                   0
ESTU_PRGM_DEPARTAMENTO                0
ESTU_VALORMATRICULAUNIVERSIDAD     6287
ESTU_HORASSEMANATRABAJA           30857
FAMI_ESTRATOVIVIENDA              32137
FAMI_TIENEINTERNET                26629
FAMI_EDUCACIONPADRE               23178
FAMI_EDUCACIONMADRE               23664
ESTU_PAGOMATRICULAPROPIO           6498
RENDIMIENTO_GLOBAL                    0
dtype: int64


#**Limpieza y preprocesamiento de datos**

**Porcentaje de valores faltantes**

In [ ]:
# Calcular el porcentaje de valores faltantes por columna
missing_percentage = data.isnull().sum() / len(data) * 100
print(missing_percentage)

# Definir un umbral del 30% para eliminar columnas con demasiados valores faltantes
threshold = 30
columns_to_drop = missing_percentage[missing_percentage > threshold].index
data_cleaned = data.drop(columns=columns_to_drop)

print(f"Columnas eliminadas: {columns_to_drop}")

ID                                0.000000
PERIODO                           0.000000
ESTU_PRGM_ACADEMICO               0.000000
ESTU_PRGM_DEPARTAMENTO            0.000000
ESTU_VALORMATRICULAUNIVERSIDAD    0.907870
ESTU_HORASSEMANATRABAJA           4.455884
FAMI_ESTRATOVIVIENDA              4.640722
FAMI_TIENEINTERNET                3.845343
FAMI_EDUCACIONPADRE               3.347004
FAMI_EDUCACIONMADRE               3.417184
ESTU_PAGOMATRICULAPROPIO          0.938339
RENDIMIENTO_GLOBAL                0.000000
dtype: float64
Columnas eliminadas: Index([], dtype='object')


**Inputar valores nulos para las columnas con valores faltantes**

In [ ]:
data['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna('Desconocido', inplace=True)
data['ESTU_HORASSEMANATRABAJA'].fillna('Desconocido', inplace=True)
data['FAMI_ESTRATOVIVIENDA'].fillna(data['FAMI_ESTRATOVIVIENDA'].mode()[0], inplace=True)
data['FAMI_TIENEINTERNET'].fillna('Desconocido', inplace=True)
data['FAMI_EDUCACIONPADRE'].fillna(data['FAMI_EDUCACIONPADRE'].mode()[0], inplace=True)
data['FAMI_EDUCACIONMADRE'].fillna(data['FAMI_EDUCACIONMADRE'].mode()[0], inplace=True)
data['ESTU_PAGOMATRICULAPROPIO'].fillna(0, inplace=True)


<ipython-input-9-4d1143fa3003>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna('Desconocido', inplace=True)
<ipython-input-9-4d1143fa3003>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, 

In [ ]:
# Explorar la cantidad de valores faltantes por columna
missing_values = data.isnull().sum()
print("Valores faltantes:\n", missing_values)

Valores faltantes:
 ID                                0
PERIODO                           0
ESTU_PRGM_ACADEMICO               0
ESTU_PRGM_DEPARTAMENTO            0
ESTU_VALORMATRICULAUNIVERSIDAD    0
ESTU_HORASSEMANATRABAJA           0
FAMI_ESTRATOVIVIENDA              0
FAMI_TIENEINTERNET                0
FAMI_EDUCACIONPADRE               0
FAMI_EDUCACIONMADRE               0
ESTU_PAGOMATRICULAPROPIO          0
RENDIMIENTO_GLOBAL                0
dtype: int64


## **Variables con orden representativo**

**Aplicar OrdinalEncoder a variables con orden representativo**

In [ ]:
# Crear un diccionario con las categorías ordenadas, ya considerando 'Desconocido'
ordinal_categories = {
    'ESTU_VALORMATRICULAUNIVERSIDAD': ['Desconocido', 'No pagó matrícula', 'Menos de 500 mil',
                                       'Entre 500 mil y menos de 1 millón', 'Entre 1 millón y menos de 2.5 millones',
                                       'Entre 2.5 millones y menos de 4 millones', 'Entre 4 millones y menos de 5.5 millones',
                                       'Entre 5.5 millones y menos de 7 millones', 'Más de 7 millones'],
    'ESTU_HORASSEMANATRABAJA': ['Desconocido', '0', 'Menos de 10 horas', 'Entre 11 y 20 horas',
                                'Entre 21 y 30 horas', 'Más de 30 horas'],
    'FAMI_ESTRATOVIVIENDA': ['Sin Estrato', 'Estrato 1', 'Estrato 2', 'Estrato 3',
                             'Estrato 4', 'Estrato 5', 'Estrato 6'],
    'RENDIMIENTO_GLOBAL': ['bajo', 'medio-bajo', 'medio-alto', 'alto']
}

# Aplicar OrdinalEncoder con manejo de valores desconocidos
ordinal_enc = OrdinalEncoder(categories=[ordinal_categories[col] for col in ordinal_categories.keys()],
                             handle_unknown='use_encoded_value', unknown_value=-1)

# Transformar las columnas que necesitan codificación ordinal
data[list(ordinal_categories.keys())] = ordinal_enc.fit_transform(data[list(ordinal_categories.keys())])

# Verificar los resultados
print(data[list(ordinal_categories.keys())].head())
print(data['ESTU_VALORMATRICULAUNIVERSIDAD'].value_counts())
print(data['ESTU_HORASSEMANATRABAJA'].value_counts())
print(data['FAMI_ESTRATOVIVIENDA'].value_counts())
print(data['RENDIMIENTO_GLOBAL'].value_counts())

   ESTU_VALORMATRICULAUNIVERSIDAD  ESTU_HORASSEMANATRABAJA  \
0                             7.0                      2.0   
1                             5.0                      1.0   
2                             5.0                      5.0   
3                             6.0                      1.0   
4                             5.0                      4.0   

   FAMI_ESTRATOVIVIENDA  RENDIMIENTO_GLOBAL  
0                   3.0                 2.0  
1                   3.0                 0.0  
2                   3.0                 0.0  
3                   4.0                 3.0  
4                   3.0                 1.0  
ESTU_VALORMATRICULAUNIVERSIDAD
4.0    204048
5.0    127430
2.0     80263
3.0     78704
6.0     69736
8.0     68014
7.0     38490
1.0     19528
0.0      6287
Name: count, dtype: int64
ESTU_HORASSEMANATRABAJA
5.0    249352
1.0    116550
3.0    115857
4.0     92693
2.0     87191
0.0     30857
Name: count, dtype: int64
FAMI_ESTRATOVIVIENDA
2.0    264808

## **Codificar variables booleanas**

In [ ]:
# Convertir las columnas booleanas a formato numérico (0 y 1)
data['ESTU_PAGOMATRICULAPROPIO'] = data['ESTU_PAGOMATRICULAPROPIO'].replace({'Si': 1, 'No': 0})

# Verificar que ahora los valores sean numéricos
data['ESTU_PAGOMATRICULAPROPIO'] = data['ESTU_PAGOMATRICULAPROPIO'].astype(int)

# Verificar los resultados
print(data[['ESTU_PAGOMATRICULAPROPIO']].head())
print(data['ESTU_PAGOMATRICULAPROPIO'].value_counts())

   ESTU_PAGOMATRICULAPROPIO
0                         0
1                         0
2                         0
3                         0
4                         0
ESTU_PAGOMATRICULAPROPIO
0    388699
1    303801
Name: count, dtype: int64


<ipython-input-12-4216059937db>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data['ESTU_PAGOMATRICULAPROPIO'] = data['ESTU_PAGOMATRICULAPROPIO'].replace({'Si': 1, 'No': 0})


## **Variables sin orden representativo**

**Agrupación de valores de 'ESTU_PRGM_DEPARTAMENTO' a otros**

In [ ]:
# Frecuencia de cada departamento
department_counts = data['ESTU_PRGM_DEPARTAMENTO'].value_counts()

# Definir el umbral
threshold = 5000  # Cambiar a 1000 si no se quiere perder representacion

# Agrupar los departamentos con menos de 'threshold' estudiantes en "Otros"
data['ESTU_PRGM_DEPARTAMENTO_AGRUPADO'] = data['ESTU_PRGM_DEPARTAMENTO'].apply(
    lambda x: x if department_counts[x] >= threshold else 'Otros'
)

# Eliminar la columna original de departamentos no agrupados
data.drop('ESTU_PRGM_DEPARTAMENTO', axis=1, inplace=True)

# Verificar los resultados
print(data['ESTU_PRGM_DEPARTAMENTO_AGRUPADO'].value_counts())


ESTU_PRGM_DEPARTAMENTO_AGRUPADO
BOGOTÁ             282159
ANTIOQUIA           83607
VALLE               44588
ATLANTICO           41020
SANTANDER           28828
NORTE SANTANDER     22588
BOLIVAR             20629
Otros               15063
BOYACA              14048
CUNDINAMARCA        14018
NARIÑO              13454
RISARALDA           12679
CORDOBA             12188
TOLIMA              11921
CALDAS              11640
CAUCA               11471
HUILA                9995
MAGDALENA            9512
SUCRE                8674
CESAR                8279
QUINDIO              8229
META                 7910
Name: count, dtype: int64


**Aplicar OneHotEncoding a variables sin orden representativo**

In [ ]:
# Definir las columnas que necesitan OneHotEncoding
columns_to_encode = ['ESTU_PRGM_DEPARTAMENTO_AGRUPADO', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE', 'FAMI_EDUCACIONMADRE']

# Crear el OneHotEncoder
onehot_enc = OneHotEncoder(drop='first', sparse_output=False)  # Eliminar la primera categoría para evitar multicolinealidad

# Aplicar el encoder a las columnas seleccionadas
encoded_columns = onehot_enc.fit_transform(data[columns_to_encode])

# Crear un DataFrame con las nuevas columnas codificadas
encoded_df = pd.DataFrame(encoded_columns, columns=onehot_enc.get_feature_names_out(columns_to_encode))

# Concatenar las nuevas columnas con el DataFrame original (eliminando las originales)
data = pd.concat([data.drop(columns_to_encode, axis=1), encoded_df], axis=1)

# Verificar los resultados
print(data.head())


       ID  PERIODO         ESTU_PRGM_ACADEMICO  \
0  904256    20212                  ENFERMERIA   
1  645256    20212                     DERECHO   
2  308367    20203       MERCADEO Y PUBLICIDAD   
3  470353    20195  ADMINISTRACION DE EMPRESAS   
4  989032    20212                  PSICOLOGIA   

   ESTU_VALORMATRICULAUNIVERSIDAD  ESTU_HORASSEMANATRABAJA  \
0                             7.0                      2.0   
1                             5.0                      1.0   
2                             5.0                      5.0   
3                             6.0                      1.0   
4                             5.0                      4.0   

   FAMI_ESTRATOVIVIENDA  ESTU_PAGOMATRICULAPROPIO  RENDIMIENTO_GLOBAL  \
0                   3.0                         0                 2.0   
1                   3.0                         0                 0.0   
2                   3.0                         0                 0.0   
3                   4.0           